Install the tweepy library

In [1]:
# !pip install tweepy
# !pip install mysqlclient

Import required libraries

In [2]:
import os
import re
import time

import tweepy as tw
import pandas as pd
from datetime import datetime, timedelta, timezone
import MySQLdb


# For sentiment analysis of tweets
from textblob import TextBlob

Create Tweeter Consumer Keys and Access Tokens 

In [3]:
consumer_key= 'NFIO7bIgwQpdCZ7pss'
consumer_secret= '2QegNrIpcGCW6n8SdIT7V0ql5cQgFwV'
access_token= '3049227557-36ln3qak9xbxHvgNos4qs'
access_token_secret= 'pZ4Y0p31graKrAYAy41mYztv'

Initialize AuthHandler

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Define search words and date

In [5]:
search_words = "#Covid"
date_since = "2018-11-16"

In [6]:
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(5)

In [7]:
tweets

Iterate through tweets using for loop

In [8]:
from datetime import datetime, timedelta
from email.utils import parsedate_tz

s='Mon Dec 07 17:03:03 +0000 2020'
def to_datetime(datestring):
    time_tuple = parsedate_tz(datestring.strip())
    dt = datetime(*time_tuple[:6])
    return dt - timedelta(seconds=time_tuple[-1])

print(to_datetime(s))
print(to_datetime(s).time().strftime("%H:%M:%S"))

x=datetime.strptime('12/31/13 00:12', "%m/%j/%y %H:%M").time().strftime("%H:%M:%S")

print(x)

2020-12-07 17:03:03
17:03:03
00:12:00


Retrieve tweets as a collection of list

In [9]:
[tweet.text for tweet in tweets]

['#COVID  \n\nMaharashtra Government \n\nAnnounces Night Curfew From 11 pm To 6 am In Municipal Corporation Areas From De… https://t.co/EGyN62cPf6',
 "RT @hall_roger: @vickydenhome The vaccine is actually worse than #Covid &amp; it's already been proven.\nANYONE with ANY known allergies (food,…",
 'RT @Tathastuaryan: All departures from #UK to #India and the other way around suspended from 23 December till 31 December, in the midst of…',
 'RT @timeshighered: Privacy fears as China keeps tracking student locations: daily check-ins launched during the #Covid outbreak appear to b…',
 'I agree, #opendata will help us move past the #COVID-19 pandemic, and help us prepare for the next health emergency… https://t.co/koWR64gQYg']

Search Tweets without Retweets

In [10]:
new_search = search_words + " -filter:retweets"

In [11]:
new_search

'#Covid -filter:retweets'

In [12]:
tweets = tw.Cursor(api.search, q=new_search, lang="en",since=date_since).items(5)

In [13]:
[tweet.text for tweet in tweets]

["Winter solstice is a great time of transition especially after dealing with energy that's not serving you after mon… https://t.co/6BzJYcnENb",
 '#COVID  \n\nMaharashtra Government \n\nAnnounces Night Curfew From 11 pm To 6 am In Municipal Corporation Areas From De… https://t.co/EGyN62cPf6',
 'I agree, #opendata will help us move past the #COVID-19 pandemic, and help us prepare for the next health emergency… https://t.co/koWR64gQYg',
 'Justin\'s drinking and drug use started increasing during college where he established himself as "the party guy". A… https://t.co/N5u3OIrhjE',
 "@MysterySolvent He is getting a nursing home patient's vaccine. Or a nurse's, doctor's, tech's who is working with… https://t.co/sU4tddA8iF"]

Get username and location

In [14]:
tweets = tw.Cursor(api.search,q=new_search, lang="en", since=date_since).items(5)

In [15]:
users_locs = [[tweet.user.screen_name,tweet.user.location,tweet.text] for tweet in tweets]

In [16]:
users_locs

[['tambraraye',
  'Washington, DC',
  "Winter solstice is a great time of transition especially after dealing with energy that's not serving you after mon… https://t.co/6BzJYcnENb"],
 ['AshVerma111',
  'Mumbai',
  '#COVID  \n\nMaharashtra Government \n\nAnnounces Night Curfew From 11 pm To 6 am In Municipal Corporation Areas From De… https://t.co/EGyN62cPf6'],
 ['JenYokoyama',
  '',
  'I agree, #opendata will help us move past the #COVID-19 pandemic, and help us prepare for the next health emergency… https://t.co/koWR64gQYg'],
 ['SteppinOutRadio',
  '',
  'Justin\'s drinking and drug use started increasing during college where he established himself as "the party guy". A… https://t.co/N5u3OIrhjE'],
 ['fl_bama',
  'Alabama, USA',
  "@MysterySolvent He is getting a nursing home patient's vaccine. Or a nurse's, doctor's, tech's who is working with… https://t.co/sU4tddA8iF"]]

Create a Pandas Dataframe From A List of Tweet Data

In [17]:
tweet_df = pd.DataFrame(data=users_locs, columns=['user', "location","tweet"])

In [18]:
tweet_df

,user,location,tweet
0,tambraraye,"Washington, DC",Winter solstice is a great time of transition ...
1,AshVerma111,Mumbai,#COVID \n\nMaharashtra Government \n\nAnnounc...
2,JenYokoyama,,"I agree, #opendata will help us move past the ..."
3,SteppinOutRadio,,Justin's drinking and drug use started increas...
4,fl_bama,"Alabama, USA",@MysterySolvent He is getting a nursing home p...


In [19]:
tweet_df.to_csv("tweets.csv")